In [113]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import matplotlib

import tensorflow as tf
from PIL import Image
import cv2
import random
from skimage.color import rgb2lab, lab2rgb

In [114]:
paths = glob("D:Dataset/dataset/coco_sample/train_sample/*")# paths 변수에 모든 이미지의 경로를 리스트로 저장.
len(paths)

21837

In [115]:
chosen_paths = np.random.choice(paths, 10000, replace=False) # 10000장의 이미지의 경로를 랜덤으로 paths_subset에 저장
index = np.random.permutation(10000) # 10000 안에서 랜덤으로 인덱스 생성

train_paths = chosen_paths[index[:8000]] # 8000번째까지를 training set 경로로 지정
val_paths = chosen_paths[index[8000:]] # 나머지 2000개를 validation set 경로로 지정
print(len(train_paths), len(val_paths))

8000 2000


In [143]:
a = np.array([1,2,3])
def time2(x):
    return 2*x
np.array(map(time2,a))

array(<map object at 0x000001C86A7B7820>, dtype=object)

In [ ]:
class ImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, train_paths, batch_size=32, train=False, size=256):
        self.paths = train_pahts
        self.dim = size
        self.train = train

    def __len__(self):
        return np.ceil(len(self.paths) / self.batch_size).astype(int)
    
    def on_epoch_end(self):
        if self.train: #Reshuffle train on end of epoch
            self.df = np.random.shuffle(self.df)
            
    def __getitem__(self, idx):
        batch_x = self.df[idx*self.batch_size:(idx+1)*self.batch_size]
        
        if not self.train:
            X = np.zeros((batch_x.shape[0], self.dim, self.dim, 3))
            
            for i in range(batch_x.shape[0]):
                image = Image.open(BASE_PATH+f"{self.pref}_images/{batch_x[i]}.tiff")
                image = image.resize((self.dim, self.dim))
                image = np.array(image) / 255.
                X[i,] = image
                
            return X
                
        else:
            batch_y = self.df.iloc[idx*self.batch_size:(idx+1)*self.batch_size].rle.values
            batch_w = self.df.iloc[idx*self.batch_size:(idx+1)*self.batch_size].img_width.values
            #print(batch_y, batch_w)
            X = np.zeros((batch_x.shape[0]*4, self.dim, self.dim, 3))
            Y = np.zeros((batch_x.shape[0]*4, self.dim, self.dim, 1))
            
            for i in range(batch_x.shape[0]):
                image = Image.open(BASE_PATH+f"{self.pref}_images/{batch_x[i]}.tiff")
                image = image.resize((self.dim, self.dim))
                image = np.array(image) / 255.
                rle = rle2mask(batch_y[i], batch_w[i], self.dim)
                rle = rle.reshape((self.dim, self.dim, 1))

                for n, (h, v) in enumerate([(0, 0), (0, 1), (1, 0), (1, 1)]):
                    X[i*4 + n, :, :, :] = self.flip_image(image, h, v)[:, :, :]
                    Y[i*4 + n, :, :, :] = self.flip_image(rle, h, v)[:, :, :]
                    
            return X, Y#.reshape(Y.shape[:-1])
                
    
    def flip_image(self, image, horizontal=True, vertical=True):
        n_image = image.copy()
        
        if horizontal:
            n_image = n_image[:, ::-1, :]
        if vertical:
            n_image = n_image[::-1, :, :]
        return n_image